# <연애 상담 챗봇(Custom_data)(unexp_rand)>

## 1. 모델/데이터 불러오기 

In [2]:
!pip install sentence_transformers

import torch
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import random

# SBERT 모델 불러오기
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# 학습된 input, output df 불러오기
data = pd.read_csv('/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_자연어처리프로젝트/application_love_chatbot_sbert_v3(custom_data)(unexp_rand)/data/csv/연애시뮬레이션(실험)_전처리.csv')
data = data[['input','output']]

# input별 embedding data 불러오기
embedding_data = torch.load('/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_자연어처리프로젝트/application_love_chatbot_sbert_v3(custom_data)(unexp_rand)/data/embedding_data/custom_love_embedding_data.pt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 38.4 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=67e639847118c9babff3b1696b677ea1d30b1abc2357db98409bbfd7dbe6979e
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


## 2. 챗봇 대화 함수 선언

In [76]:
# 가장 최근 input
last_sentence = ''

# 가장 최근 출력 idx
last_sim_idx_list = []

# 현재 중복 cnt
overlap_cnt = 0

# 학습에 없는 질문 리스트
unexpected_input_list = []

def talk_to_bot(sentence):

  global last_sentence 
  global last_sim_idx_list
  global overlap_cnt
  global unexpected_input






  # 알아듣는 문장 유사도 임계값
  output_thresholds = 0.7

  # 중복 문장 유사도 임계값
  overlap_thresholds = 0.85
  remove_sim_idx = None

  sentence = sentence.replace(" ","")
  sentence_encode = model.encode(sentence)
  sentence_tensor = torch.tensor(sentence_encode)

  last_sentence = last_sentence.replace(" ","")
  last_sentence_encode = model.encode(last_sentence)
  last_sentence_tensor = torch.tensor(last_sentence_encode)

  overlap_cos_sim = util.cos_sim(sentence_tensor, last_sentence_tensor)

  cos_sim = util.cos_sim(sentence_tensor, embedding_data)






  rand_num1 = random.random()
  # 예상 외의 input 입력 시
  if cos_sim[0][int(np.argmax(cos_sim))] < output_thresholds:
    print(f'응애 나 아기 챗봇!\n문장 유사도 : {round(cos_sim[0][int(np.argmax(cos_sim))],2)}')
    print(f'\n새로운 데이터로 아기 챗봇 ({round(rand_num1*50,1)}%) 성장!')
    unexpected_input_list.append(sentence)
    
    return last_sentence, last_sim_idx_list , overlap_cnt
    

  # 이전 질문과 문장이 0.85이상 유사하면 
  # 해당 질문-답변 출력 X
  # 총 3번까지
  if overlap_cos_sim >= overlap_thresholds and overlap_cnt < 3 :
    print('************************ 이전 질문과 유사함 ************************')
    remove_sim_idx_list = last_sim_idx_list

    for i in remove_sim_idx_list:
      cos_sim[0][int(i)] = 0
    overlap_cnt += 1
    print(f'이전 출력 idx: {last_sim_idx_list}')
    print(f'이전 질문과 유사도: {overlap_cos_sim}')
    print(f'중복횟수 : {overlap_cnt}')
  else:
    overlap_cnt = 0
    last_sim_idx_list = []





  print(f'사용자의 질문: {sentence} \n')

  dot_score = util.dot_score(sentence_tensor, embedding_data)
  ascending_cos_sim = np.sort(cos_sim)
  descending_cos_sim = ascending_cos_sim[::-1]
  # print(f'cos_sim 유사도 내림차순: {ascending_cos_sim}')

  print(f'--------------------------- 최우선 순위 질문/답변 ---------------------------')
  print(f"가장 높은 코사인 유사도 idx : {int(np.argmax(cos_sim))}")





  # 동일한 문장 유사도 input 일 때, 랜덤으로 출력

  max_cos_sim = cos_sim[0][np.argmax(cos_sim)]
  same_cos_sim_list = []

  for idx,one_cos_sim in enumerate(cos_sim[0]):
    if one_cos_sim == max_cos_sim:
      same_cos_sim_list.append(idx)

  print(f'same_cos_sim_list 크기: {len(same_cos_sim_list)}')
  if len(same_cos_sim_list) == 1:
    # data에서 선택된 질문 출력
    best_sim_idx = int(np.argmax(cos_sim))
    sentence_qes = data['input'][best_sim_idx]
    print(f"선택된 질문 = {sentence_qes}")

    # data에서 선택된 질문 문장에 대한 인코딩
    selected_qes_encode = model.encode(sentence_qes)

    # 유사도 점수 측정
    # print(f'np.dot(sentence_tensor, selected_qes_encode) 값: {np.dot(sentence_tensor, sentence_encode)}')
    # print(f'(np.linalg.norm(sentence_tensor) 값: {np.linalg.norm(sentence_tensor)}')
    # print(f'(np.linalg.norm(selected_qes_encode) 값: {np.linalg.norm(selected_qes_encode)}')
    print(f'util.cos_sim 활용 코사인 유사도 : {cos_sim[0][best_sim_idx]}')
    score = np.dot(sentence_tensor, sentence_encode) / (np.linalg.norm(sentence_tensor) * np.linalg.norm(selected_qes_encode))
    print(f"직접 수식 작성한 코사인 유사도 = {score}")

    # 답변
    answer = data['output'][best_sim_idx]
    print(f"\n답변 : {answer}\n")

    # 최근 출력한 idx 기억
    last_sim_idx_list.append(int(np.argmax(cos_sim)))
  
  else:
    rand_num2 = random.choice(same_cos_sim_list)
    print(f'같은 문장 유사도 list:{same_cos_sim_list}')
    print(f'rand_num2 뽑음!')
    print(rand_num2)
    
    # data에서 선택된 질문 출력
    best_sim_idx = rand_num2
    sentence_qes = data['input'][best_sim_idx]
    print(f"선택된 질문 = {sentence_qes}")

    # data에서 선택된 질문 문장에 대한 인코딩
    selected_qes_encode = model.encode(sentence_qes)

    # 유사도 점수 측정
    # print(f'np.dot(sentence_tensor, selected_qes_encode) 값: {np.dot(sentence_tensor, sentence_encode)}')
    # print(f'(np.linalg.norm(sentence_tensor) 값: {np.linalg.norm(sentence_tensor)}')
    # print(f'(np.linalg.norm(selected_qes_encode) 값: {np.linalg.norm(selected_qes_encode)}')
    print(f'util.cos_sim 활용 코사인 유사도 : {cos_sim[0][best_sim_idx]}')
    score = np.dot(sentence_tensor, sentence_encode) / (np.linalg.norm(sentence_tensor) * np.linalg.norm(selected_qes_encode))
    print(f"직접 수식 작성한 코사인 유사도 = {score}")

    # 답변
    answer = data['output'][best_sim_idx]
    print(f"\n답변 : {answer}\n")

    # 최근 출력한 idx 기억
    last_sim_idx_list.append(int(best_sim_idx))







  # 중복 input 대응을 위해 이전sentence 저장 
  last_sentence = sentence

  # 최우선 순위 질문 벡터 초기화 
  cos_sim[0][best_sim_idx] = 0








  # 후순위 답변/질문 디버깅
  print(f'--------------------------- 후순위 답변/질문 디버깅 ---------------------------')
  for i in range(1,6):
    print(f'<{i}순위 Case>')
    print(f"가장 높은 코사인 유사도 idx : {int(np.argmax(cos_sim))}")

    # data에서 선택된 질문 출력
    best_sim_idx = int(np.argmax(cos_sim))
    sentence_qes = data['input'][best_sim_idx]
    print(f"선택된 질문 = {sentence_qes}")

    # data에서 선택된 질문 문장에 대한 인코딩
    selected_qes_encode = model.encode(sentence_qes)

    # 유사도 점수 측정
    # print(f'np.dot(sentence_tensor, selected_qes_encode) 값: {np.dot(sentence_tensor, sentence_encode)}')
    # print(f'(np.linalg.norm(sentence_tensor) 값: {np.linalg.norm(sentence_tensor)}')
    # print(f'(np.linalg.norm(selected_qes_encode) 값: {np.linalg.norm(selected_qes_encode)}')
    print(f'util.cos_sim 활용 코사인 유사도 : {cos_sim[0][best_sim_idx]}')
    score = np.dot(sentence_tensor, sentence_encode) / (np.linalg.norm(sentence_tensor) * np.linalg.norm(selected_qes_encode))
    print(f"직접 수식 작성한 코사인 유사도 = {score}")

    # 답변
    answer = data['output'][best_sim_idx]
    print(f"\n답변 : {answer}\n")

    # 최우선 순위 질문 벡터 초기화 
    cos_sim[0][best_sim_idx] = 0

  print(f'-------------------------------------------------------------------------------')

  return last_sentence, last_sim_idx_list , overlap_cnt

## 3. 챗봇 실행 
## ex) talk_to_bot('ㅎㅇ')

In [65]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('연인이랑 무슨 얘기해?')

사용자의 질문: 연인이랑무슨얘기해? 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 9
same_cos_sim_list 크기: 9
같은 문장 유사도 list:[9, 10, 11, 12, 13, 14, 15, 16, 17]
rand_num2 뽑음!
16
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 요즘 유행하는 주제에 대해서 한 번 물어 봐

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 9
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 같은 취미를 가져보는 건 어때? 공통 화제가 있으면 대화가 잘 통할 거야.

<2순위 Case>
가장 높은 코사인 유사도 idx : 10
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 일상 얘기부터 해봐~

<3순위 Case>
가장 높은 코사인 유사도 idx : 11
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 목소리만 들어도 좋잖아, 어떤 주제라도 이야기가 잘 통할거야.

<4순위 Case>
가장 높은 코사인 유사도 idx :

In [66]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('연인이랑 무슨 얘기해?')

************************ 이전 질문과 유사함 ************************
이전 출력 idx: [16]
이전 질문과 유사도: tensor([[1.]])
중복횟수 : 1
사용자의 질문: 연인이랑무슨얘기해? 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 9
same_cos_sim_list 크기: 8
같은 문장 유사도 list:[9, 10, 11, 12, 13, 14, 15, 17]
rand_num2 뽑음!
11
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 목소리만 들어도 좋잖아, 어떤 주제라도 이야기가 잘 통할거야.

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 9
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 같은 취미를 가져보는 건 어때? 공통 화제가 있으면 대화가 잘 통할 거야.

<2순위 Case>
가장 높은 코사인 유사도 idx : 10
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 일상 얘기부터 해봐~

<3순위 Case>
가장 높은 코사인 유사도 idx : 12
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.799359560

In [67]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('연인이랑 무슨 얘기해?')

************************ 이전 질문과 유사함 ************************
이전 출력 idx: [16, 11]
이전 질문과 유사도: tensor([[1.]])
중복횟수 : 2
사용자의 질문: 연인이랑무슨얘기해? 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 9
same_cos_sim_list 크기: 7
같은 문장 유사도 list:[9, 10, 12, 13, 14, 15, 17]
rand_num2 뽑음!
13
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 오늘 하루 어땠는지 얘기해 봐.

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 9
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 같은 취미를 가져보는 건 어때? 공통 화제가 있으면 대화가 잘 통할 거야.

<2순위 Case>
가장 높은 코사인 유사도 idx : 10
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 일상 얘기부터 해봐~

<3순위 Case>
가장 높은 코사인 유사도 idx : 12
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한

In [68]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('연인이랑 무슨 얘기해?')

************************ 이전 질문과 유사함 ************************
이전 출력 idx: [16, 11, 13]
이전 질문과 유사도: tensor([[1.]])
중복횟수 : 3
사용자의 질문: 연인이랑무슨얘기해? 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 9
same_cos_sim_list 크기: 6
같은 문장 유사도 list:[9, 10, 12, 14, 15, 17]
rand_num2 뽑음!
9
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 같은 취미를 가져보는 건 어때? 공통 화제가 있으면 대화가 잘 통할 거야.

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 10
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 일상 얘기부터 해봐~

<2순위 Case>
가장 높은 코사인 유사도 idx : 12
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 데이트 계획 같이 세워 봐~! 같이 하고싶은 게 많잖아

<3순위 Case>
가장 높은 코사인 유사도 idx : 14
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128

In [69]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('연인이랑 무슨 얘기해?')

사용자의 질문: 연인이랑무슨얘기해? 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 9
same_cos_sim_list 크기: 9
같은 문장 유사도 list:[9, 10, 11, 12, 13, 14, 15, 16, 17]
rand_num2 뽑음!
17
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 관심사에 대해 물어 봐

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 9
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 같은 취미를 가져보는 건 어때? 공통 화제가 있으면 대화가 잘 통할 거야.

<2순위 Case>
가장 높은 코사인 유사도 idx : 10
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 일상 얘기부터 해봐~

<3순위 Case>
가장 높은 코사인 유사도 idx : 11
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 목소리만 들어도 좋잖아, 어떤 주제라도 이야기가 잘 통할거야.

<4순위 Case>
가장 높은 코사인 유사도 idx : 12
선택된 질문 =

In [31]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('연인이랑 무슨 얘기해?')

************************ 이전 질문과 유사함 ************************
이전 출력 idx: [9, 10, 11]
이전 질문과 유사도: tensor([[1.]])
중복횟수 : 3
사용자의 질문: 연인이랑무슨얘기해? 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 12
6
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 내가 이런 것 까지 가르쳐 줘야 하는거야~? ㅋㅋㅋㅋㅋ

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 12
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 데이트 계획 같이 세워 봐~! 같이 하고싶은 게 많잖아

<2순위 Case>
가장 높은 코사인 유사도 idx : 13
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 오늘 하루 어땠는지 얘기해 봐.

<3순위 Case>
가장 높은 코사인 유사도 idx : 15
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 평소에 많이 얘기 해서 할 말이 없다면 굳이 전화하지 않아도

In [32]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('연인이랑 무슨 얘기해?')

사용자의 질문: 연인이랑무슨얘기해? 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 9
9
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 관심사에 대해 물어 봐

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 9
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 같은 취미를 가져보는 건 어때? 공통 화제가 있으면 대화가 잘 통할 거야.

<2순위 Case>
가장 높은 코사인 유사도 idx : 10
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 일상 얘기부터 해봐~

<3순위 Case>
가장 높은 코사인 유사도 idx : 11
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.9482440948486328

답변 : 목소리만 들어도 좋잖아, 어떤 주제라도 이야기가 잘 통할거야.

<4순위 Case>
가장 높은 코사인 유사도 idx : 12
선택된 질문 = 연인이랑 전화할 때 무슨 얘기 해?
util.cos_sim 활용 코사인 유사도 : 0.7993595600128174
직접 수식 작성한 코사인 유사도 = 0.94

In [74]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('연락을 할까 말까')

응애 나 아기 챗봇!
문장 유사도 : 0.5490211844444275

새로운 데이터로 아기 챗봇 (43.4%) 성장!


In [75]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('약속 시간에 늦을까 안늦을까')

응애 나 아기 챗봇!
문장 유사도 : 0.2798125743865967

새로운 데이터로 아기 챗봇 (12.7%) 성장!


In [77]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('여자친구 선물로 뭐가 좋을까')

사용자의 질문: 여자친구선물로뭐가좋을까 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 519
same_cos_sim_list 크기: 5
같은 문장 유사도 list:[519, 520, 521, 522, 523]
rand_num2 뽑음!
523
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276361465454
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 비싼 물건보다 평소 그 사람이 갖고싶어 했거나, 필요로 한 물건을 주는게 더 감동일 수 있어

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 519
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276361465454
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 상대방이 평소에 갖고 싶어했던 물건이 있어?

<2순위 Case>
가장 높은 코사인 유사도 idx : 520
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276361465454
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 인터넷에 보면 많이 추천하는 선물 목록이 있던데~

<3순위 Case>
가장 높은 코사인 유사도 idx : 521
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276361465454
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 화장품이나 향수처럼 취향을 타는 선물은 고민 많이 하고 골라야 해

<4순

In [78]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('여자친구 선물로 뭐가 좋을까')

************************ 이전 질문과 유사함 ************************
이전 출력 idx: [523]
이전 질문과 유사도: tensor([[1.0000]])
중복횟수 : 1
사용자의 질문: 여자친구선물로뭐가좋을까 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 519
same_cos_sim_list 크기: 4
같은 문장 유사도 list:[519, 520, 521, 522]
rand_num2 뽑음!
522
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276361465454
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 요즘엔 쓸모없는 선물 주는 것도 유행이라던데? 어때? ㅎㅎㅎ

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 519
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276361465454
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 상대방이 평소에 갖고 싶어했던 물건이 있어?

<2순위 Case>
가장 높은 코사인 유사도 idx : 520
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276361465454
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 인터넷에 보면 많이 추천하는 선물 목록이 있던데~

<3순위 Case>
가장 높은 코사인 유사도 idx : 521
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.76642763

In [79]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('여자친구 선물로 뭐가 좋을까')

************************ 이전 질문과 유사함 ************************
이전 출력 idx: [523, 522]
이전 질문과 유사도: tensor([[1.0000]])
중복횟수 : 2
사용자의 질문: 여자친구선물로뭐가좋을까 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 519
same_cos_sim_list 크기: 3
같은 문장 유사도 list:[519, 520, 521]
rand_num2 뽑음!
521
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276361465454
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 화장품이나 향수처럼 취향을 타는 선물은 고민 많이 하고 골라야 해

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 519
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276361465454
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 상대방이 평소에 갖고 싶어했던 물건이 있어?

<2순위 Case>
가장 높은 코사인 유사도 idx : 520
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276361465454
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 인터넷에 보면 많이 추천하는 선물 목록이 있던데~

<3순위 Case>
가장 높은 코사인 유사도 idx : 529
선택된 질문 = 남자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.70825

In [80]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('여자친구 선물로 뭐가 좋을까')

************************ 이전 질문과 유사함 ************************
이전 출력 idx: [523, 522, 521]
이전 질문과 유사도: tensor([[1.0000]])
중복횟수 : 3
사용자의 질문: 여자친구선물로뭐가좋을까 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 519
same_cos_sim_list 크기: 2
같은 문장 유사도 list:[519, 520]
rand_num2 뽑음!
519
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276361465454
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 상대방이 평소에 갖고 싶어했던 물건이 있어?

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 520
선택된 질문 = 여자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7664276361465454
직접 수식 작성한 코사인 유사도 = 0.9644625782966614

답변 : 인터넷에 보면 많이 추천하는 선물 목록이 있던데~

<2순위 Case>
가장 높은 코사인 유사도 idx : 529
선택된 질문 = 남자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7082570791244507
직접 수식 작성한 코사인 유사도 = 0.9903244972229004

답변 : 상대방이 평소에 갖고 싶어했던 물건이 있어?

<3순위 Case>
가장 높은 코사인 유사도 idx : 530
선택된 질문 = 남자친구한테 선물 주려는데 추천 좀
util.cos_sim 활용 코사인 유사도 : 0.7082570791244507
